In [17]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *
import random


# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 10
tiles_vert = 7

antal_celler  = 7
start_energi = 10

# Create empty world, to contain references to cells
verd = []
for x in range(tiles_hori):
    row = []
    for y in range(tiles_vert):
        row.append(None)
    verd.append(row)

# Populate world
for i in range(0, antal_celler):
    x = random.randint(0, tiles_hori - 1)
    y = random.randint(0, tiles_vert - 1)
    while verd[x][y] != None and verd:
        print(x, y, verd[x][y])
        x = random.randint(0, tiles_hori - 1)
        y = random.randint(0, tiles_vert - 1)
    print(x, y)
    verd[x][y] = Celle(x, y, start_energi, [])
    for _x in verd:
        print(_x)

# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)

def draw_world():
    # Draw grid
    canvas.fill_style = "gray"
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)


with hold_canvas():
    draw_world()

2 0
[None, None, None]
[None, None, None]
[<Klasser.celle.Celle object at 0x0000027C24226F20>, None, None]
0 1
[None, <Klasser.celle.Celle object at 0x0000027C332DBD90>, None]
[None, None, None]
[<Klasser.celle.Celle object at 0x0000027C24226F20>, None, None]
0 0
[<Klasser.celle.Celle object at 0x0000027C332DBD30>, <Klasser.celle.Celle object at 0x0000027C332DBD90>, None]
[None, None, None]
[<Klasser.celle.Celle object at 0x0000027C24226F20>, None, None]
2 2
[<Klasser.celle.Celle object at 0x0000027C332DBD30>, <Klasser.celle.Celle object at 0x0000027C332DBD90>, None]
[None, None, None]
[<Klasser.celle.Celle object at 0x0000027C24226F20>, None, <Klasser.celle.Celle object at 0x0000027C332DBD60>]
2 1
[<Klasser.celle.Celle object at 0x0000027C332DBD30>, <Klasser.celle.Celle object at 0x0000027C332DBD90>, None]
[None, None, None]
[<Klasser.celle.Celle object at 0x0000027C24226F20>, <Klasser.celle.Celle object at 0x0000027C332DBE50>, <Klasser.celle.Celle object at 0x0000027C332DBD60>]
1 2
[

Canvas(height=45, width=45)